In [11]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [12]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [13]:
input_folder = '../data/railways/Amtrak_Routes/'
input_file = input_folder+'Amtrak_Routes.shp'

output_folder = '../data/output_images/LosAngelesMR/'
output_csv = output_folder+'LAA.csv'

update_folder = output_folder+'/set_2/'

### Import Routes

In [3]:
blue = gpd.read_file('LAMR/801_Blue_Track/801_Track_0316.shp')
blue = blue.to_crs({'init' :'epsg:4269'})

red = gpd.read_file('LAMR/802_Red_Purple_Track/802_805_Track_0316.shp')
red = red.to_crs({'init' :'epsg:4269'})

green = gpd.read_file('LAMR/803_Green_Track/803_Track_0316.shp')
green = green.to_crs({'init' :'epsg:4269'})

gold = gpd.read_file('LAMR/804_Gold_Track/804_Track_0316.shp')
gold = gold.to_crs({'init' :'epsg:4269'})

expo = gpd.read_file('LAMR/806_Expo_Track/806_Track_0316.shp')
expo = expo.to_crs({'init' :'epsg:4269'})

frames = [blue,red,green,gold,expo]

routes = gpd.GeoDataFrame(pd.concat(frames, ignore_index=True) )

routes.head()

/Users/jessedecker/miniconda3/envs/geo1/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Id,OBJECTID,SHAPE_Leng,geometry
0,0.0,NaN,NaN,LINESTRING (-118.1893707098661 33.777799051058...
1,0.0,NaN,NaN,LINESTRING (-118.1937438467048 33.776128475602...
2,0.0,NaN,NaN,LINESTRING (-118.1937438467048 33.776128475602...
3,0.0,NaN,NaN,LINESTRING (-118.1937425194715 33.768454143462...
4,0.0,NaN,NaN,LINESTRING (-118.1929738864244 33.767995720331...


In [4]:
routes.shape

(480, 4)

In [7]:
'''
Get all points
'''

points = []
for i in range(0,len(routes.index.tolist())):
    try:
        route = routes.iloc[i]
        g = [m for m in route.geometry]
        for j in range(0,len(g)):
            sec_points = mapping(g[j])["coordinates"]
            for k in range(0,len(sec_points)):
                points.append(sec_points[k])
    except Exception as e:
        print("Skipped route number",i,'because',e)

print(len(points),'points')

Skipped route number 0 because 'LineString' object is not iterable
Skipped route number 1 because 'LineString' object is not iterable
Skipped route number 2 because 'LineString' object is not iterable
Skipped route number 3 because 'LineString' object is not iterable
Skipped route number 4 because 'LineString' object is not iterable
Skipped route number 5 because 'LineString' object is not iterable
Skipped route number 6 because 'LineString' object is not iterable
Skipped route number 7 because 'LineString' object is not iterable
Skipped route number 8 because 'LineString' object is not iterable
Skipped route number 9 because 'LineString' object is not iterable
Skipped route number 10 because 'LineString' object is not iterable
Skipped route number 11 because 'LineString' object is not iterable
Skipped route number 12 because 'LineString' object is not iterable
Skipped route number 13 because 'LineString' object is not iterable
Skipped route number 14 because 'LineString' object is not

In [5]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

Skipped route number 0 because 'LineString' object is not iterable
0 points


In [6]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

Skipped route number 0 because 'LineString' object is not iterable
0 points


In [8]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),38):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points))

50


In [9]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]


df.head()

,Name,Longitude,Latitude
0,-118.25863629843894_34.04870380692666,-118.258636,34.048704
1,-118.2916031689273_34.0695305562798,-118.291603,34.069531
2,-118.29512902367632_34.10181974819955,-118.295129,34.101820
3,-118.3472986187191_34.10478898684855,-118.347299,34.104789
4,-118.3622037256695_34.14191786748499,-118.362204,34.141918


In [10]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [14]:
'''
Open csv
'''

df = pd.read_csv(output_csv)
df.head()

,Name,Longitude,Latitude,Catenary
0,-117.99040536288051_34.13243703485112,-117.990405,34.132437,1
1,-117.99124413162227_34.134573948903366,-117.991244,34.134574,1
2,-117.99138900131467_34.13427676043347,-117.991389,34.134277,1
3,-117.991604219868_34.13426232513817,-117.991604,34.134262,1
4,-117.99180070681261_34.13442129341215,-117.991801,34.134421,1


In [9]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))

# Set map centerpoint
coords = marker_points[int((len(marker_points)/2))]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 7
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [108]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [15]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,2):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 